# scikit-learnとXGBoostの使い方

## 最初の見取り図

scikit-learnとXGBoostの使い方では、式を暗記するより、計算の意味をコードで検証する姿勢を優先します。

前提: Pythonの基本文法と、平均・分散・相関の直感が前提です。

到達目標: 学習データの準備から評価指標の読み解きまで、モデル改善の往復を自力で回せる状態にします。

ここで扱う中心語は 「特徴量」、「目的変数」、「損失関数」、「汎化」、「過学習」、「scikit-learnとXGBoostの使い方」 です。用語を先に暗記するのではなく、コード実行の結果と結びつけて理解します。

このノートは 機械学習 分野の初学者向けに、説明とコードを交互に読み進める設計です。最初から完璧に理解する必要はありません。大切なのは、各コードの目的を一文で言えることと、出力が変わる理由を自分で確かめることです。

## 手を動かす 1: scikit-learn最小実行

ライブラリを使った実装は中身理解の後に行います。ここでは API の最小単位を確認します。

In [ ]:
try:
    from sklearn.linear_model import LinearRegression
except ModuleNotFoundError:
    print('scikit-learn is not installed')
else:
    X = [[1], [2], [3], [4]]
    y = [2, 4, 6, 8]
    model = LinearRegression().fit(X, y)
    print('coef =', model.coef_[0], 'intercept =', model.intercept_)

API を最小例で確認してから本番データへ移ると、デバッグの切り分けがしやすくなります。

特徴量 を1つ変更して、同じコードを別条件で再実行し、差分の理由を1段落でまとめてください。

## 手を動かす 2: 単回帰を手で計算する

次に、最小二乗法の係数を手計算で求めます。既存ライブラリを使う前に中身を一度体験すると理解が安定します。

In [ ]:
x_bar = sum(x) / len(x)
y_bar = sum(y) / len(y)
num = sum((xi - x_bar) * (yi - y_bar) for xi, yi in pairs)
den = sum((xi - x_bar) ** 2 for xi in x)
w1 = num / den; w0 = y_bar - w1 * x_bar
print('w0, w1 =', round(w0, 4), round(w1, 4))

式の記号は抽象的に見えますが、コードでは平均・差分・和に分解されます。難しい式ほど、実装で部品に分けて確認するのが有効です。

特徴量 を1つ変更して、同じコードを別条件で再実行し、差分の理由を1段落でまとめてください。

## 数式メモ

1. y_hat = f_theta(x)
2. L(theta) = (1/N) * sum_i l(y_hat_i, y_i)

## 手を動かす 3: 予測と誤差を見る

ここで、求めた係数で予測を出し、誤差を数値化します。モデル改善は誤差の観察から始まります。

In [ ]:
pred = [w0 + w1 * xi for xi in x]
residual = [yi - pi for yi, pi in zip(y, pred)]
mse = sum((r ** 2) for r in residual) / len(residual)
print('pred =', pred)
print('mse =', round(mse, 8))

誤差を観察するときは、平均値だけでなく個別の残差も見てください。偏りがあると、モデル構造の見直しが必要になります。

特徴量 を1つ変更して、同じコードを別条件で再実行し、差分の理由を1段落でまとめてください。

## 手を動かす 4: 特徴量を拡張する

次に、特徴量を追加したときに表現力がどう変わるかを確認します。特徴量設計は精度に直接効く実装作業です。

In [ ]:
x2 = [xi ** 2 for xi in x]
feature = list(zip(x, x2))
print('feature sample =', feature[:3])
scaled_x = [(xi - x_bar) / (max(x) - min(x)) for xi in x]
print('scaled_x =', [round(v, 4) for v in scaled_x])

特徴量を増やすと表現力は上がりますが、同時に過学習リスクも増えます。増やす理由を説明できる特徴だけを採用する姿勢が重要です。

特徴量 を1つ変更して、同じコードを別条件で再実行し、差分の理由を1段落でまとめてください。

## 手を動かす 5: 評価分割を体験する

最後に、学習と評価を分ける意味をコードで確かめます。分割しない評価は自己採点に近く、実運用の性能を過大評価しがちです。

In [ ]:
train_x, test_x = x[:3], x[3:]
train_y, test_y = y[:3], y[3:]
pred_test = [w0 + w1 * xi for xi in test_x]
mae_test = sum(abs(yi - pi) for yi, pi in zip(test_y, pred_test)) / len(test_y)
print('test mae =', round(mae_test, 6))

検証データでの誤差が急に悪化したら、モデルより先にデータ分布の差を疑ってください。この視点は実務で非常に重要です。

特徴量 を1つ変更して、同じコードを別条件で再実行し、差分の理由を1段落でまとめてください。

## 最後のチェック

1. 訓練データと評価データの分布差を見ない
2. 単一スコアだけでモデルを選んでしまう
3. 前処理の漏れを評価後に気づく

思考実験: 訓練誤差が小さいのに本番誤差が大きいとき、どの仮定が崩れているかを考えます。

同じ問題を別の実装で解き、何が本質かを比較してください。

次は「特徴量エンジニアリング」へ進み、今回のコードと何が変わるかを比較してください。